In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
#Просто массив и его мини-анализ
array = np.random.randint(0,3, 20)
print(array)
# в numpy почему-то нет моды
print('mode:', stats.mode(array))
print('median:', np.median(array))
print('mean:', np.mean(array))

[2 1 0 1 1 0 2 0 1 0 0 1 1 0 0 0 0 2 0 0]
mode: ModeResult(mode=0, count=11)
median: 0.0
mean: 0.6


# Задания с демо-курса по статистике

In [5]:
#Поиграем с датасетом с курса по статистике

taxi = pd.read_csv("Datasets/2_taxi_nyc.csv")

In [12]:
taxi.shape # Узнаем кол-во строк и столбцов

(29101, 14)

In [13]:
taxi.dtypes # Узнаем типы данных в сете

pickup_dt        object
pickup_month     object
borough          object
pickups           int64
hday             object
spd             float64
vsb             float64
temp            float64
dewp            float64
slp             float64
pcp 01          float64
pcp 06          float64
pcp 24          float64
sd              float64
dtype: object

In [14]:
taxi.head(10)

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
5,2015-01-01 01:00:00,Jan,Staten Island,6,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
6,2015-01-01 01:00:00,Jan,NaN,4,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
7,2015-01-01 02:00:00,Jan,Bronx,120,Y,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0
8,2015-01-01 02:00:00,Jan,Brooklyn,1229,Y,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0
9,2015-01-01 02:00:00,Jan,EWR,0,Y,3.0,10.0,30.0,6.0,1023.0,0.0,0.0,0.0,0.0


In [17]:
# У нас есть неудобные колонки меняем их названия
taxi = taxi.rename(columns={'pcp 01':'pcp_01','pcp 06':'pcp_06','pcp 24':'pcp_24'})

In [16]:
taxi.columns

Index(['pickup_dt', 'pickup_month', 'borough', 'pickups', 'hday', 'spd', 'vsb',
       'temp', 'dewp', 'slp', 'pcp_01', 'pcp_06', 'pcp_24', 'sd'],
      dtype='object')

In [19]:
# Узнаем сколько раз встречаются районы разными способами

taxi.query("borough == 'Brooklyn'").shape[0]

4343

In [20]:
taxi['borough'].value_counts()

borough
Bronx            4343
Brooklyn         4343
EWR              4343
Manhattan        4343
Queens           4343
Staten Island    4343
Name: count, dtype: int64

In [22]:
#Узнаем общее число поездок
taxi['pickups'].sum()

14265773

In [25]:
taxi.groupby('borough').agg({'pickups':'sum'}) #Узнаем число поездок по районам

,pickups
borough,
Bronx,220047
Brooklyn,2321035
EWR,105
Manhattan,10367841
Queens,1343528
Staten Island,6957


In [29]:
#Ускоряем ответ - сделаем так, чтобы минимальные поездки нам сами вывелись
min_pickups = taxi.groupby('borough').agg({'pickups':'sum'}).idxmin()
print(min_pickups)

pickups    EWR
dtype: object


In [31]:
#Район города и выходной день
taxi.query("hday == 'Y'").groupby('borough').agg({'pickups':'sum'})

,pickups
borough,
Bronx,8027
Brooklyn,88011
EWR,7
Manhattan,340000
Queens,53562
Staten Island,250


In [37]:
taxi.query("hday == 'N'").groupby('borough').agg({'pickups':'sum'})

,pickups
borough,
Bronx,212020
Brooklyn,2233024
EWR,98
Manhattan,10027841
Queens,1289966
Staten Island,6707


In [38]:
taxi.query("hday == 'Y'").groupby('borough').agg({'pickups':'sum'})

,pickups
borough,
Bronx,8027
Brooklyn,88011
EWR,7
Manhattan,340000
Queens,53562
Staten Island,250


In [49]:
#Задание такое, надо определить в каких районах в выходные дни больше закозов
grouped_taxi = taxi.groupby(['borough', 'hday']).agg({'pickups': 'mean'}).reset_index()
grouped_taxi

,borough,hday,pickups
0,Bronx,N,50.771073
1,Bronx,Y,48.065868
2,Brooklyn,N,534.727969
3,Brooklyn,Y,527.011976
4,EWR,N,0.023467
5,EWR,Y,0.041916
6,Manhattan,N,2401.302921
7,Manhattan,Y,2035.928144
8,Queens,N,308.899904
9,Queens,Y,320.730539


Для каждого района посчитайте число поездок по месяцам. Отсортируйте полученные значения по убыванию и сохраните результирующий датафрейм в pickups_by_mon_bor.

In [57]:
pickups_by_mon_bor = taxi.groupby(['pickup_month','borough']).agg({'pickups':'sum'}).sort_values('pickups',ascending = False).reset_index()

Теперь задача посложнее! Попрактикуемся в написании собственных функций и их применении к датафрейму. Поскольку данные о поездках в Нью-Йорке, температура представлена в градусах Фаренгейта. 

Напишите функцию temp_to_celcius, которая получает на вход колонку с температурой в °F и возвращает значения, переведённые в градусы Цельсия.

In [59]:
def temp_to_celcius(temp_series):
    return (temp_series - 32)*5/9

In [61]:
# Применяем функцию к колонке 'temp'
taxi['temp_celsius'] = temp_to_celcius(taxi['temp'])

# Выводим результат
print(taxi[['temp', 'temp_celsius']].head())

   temp  temp_celsius
0  30.0     -1.111111
1  30.0     -1.111111
2  30.0     -1.111111
3  30.0     -1.111111
4  30.0     -1.111111


# Начало работы с Titanic и pandas

In [20]:
df = pd.read_csv("Datasets/titanic.csv")

In [21]:
df.head(10)

,Passengerid,Age,Fare,Sex,sibsp,zero,zero.1,zero.2,zero.3,zero.4,...,zero.12,zero.13,zero.14,Pclass,zero.15,zero.16,Embarked,zero.17,zero.18,2urvived
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0.0,0,0,1
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,1
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,1,0,0,2.0,0,0,1
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
5,6,28.0,8.4583,0,0,0,0,0,0,0,...,0,0,0,3,0,0,1.0,0,0,0
6,7,54.0,51.8625,0,0,0,0,0,0,0,...,0,0,0,1,0,0,2.0,0,0,0
7,8,2.0,21.0750,0,3,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
8,9,27.0,11.1333,1,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,1
9,10,14.0,30.0708,1,1,0,0,0,0,0,...,0,0,0,2,0,0,0.0,0,0,1
